<a href="https://colab.research.google.com/github/jiaoyao-1996/micro-expression-recognition/blob/master/U1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/miro"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['main.py',
 'happy',
 'disgust',
 'angry',
 '.idea',
 'microexpstcnn_images.npy',
 'microexpstcnn_labels.npy',
 'weights-improvement-53-0.88.hdf5',
 'microexpstcnn_val_images.npy',
 'microexpstcnn_val_labels.npy']

In [111]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 这一行注释掉就是使用gpu，不注释就是使用cpu
import cv2
import numpy
import imageio
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from tensorflow.keras import backend as K
import sys
from keras.models import load_model
import tensorflow as tf



In [112]:
K.set_image_data_format('channels_last')

image_rows, image_columns, image_depth = 64, 64, 41


training_list = []
angrypath = './angry/'
happypath = './happy/'
disgustpath = './disgust/'


In [113]:
directorylisting = os.listdir(angrypath)
for video in directorylisting:
	frames = []
	videopath = angrypath + video
	framerange = os.listdir(videopath)

	for frame in framerange:
		image = cv2.imread(videopath + "/" + frame)
		imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
		grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
		frames.append(grayimage)
	frames = numpy.asarray(frames)
	videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
	training_list.append(videoarray)

In [114]:
directorylisting = os.listdir(happypath)
for video in directorylisting:
	frames = []
	videopath = happypath + video
	framerange = os.listdir(videopath)

	for frame in framerange:
		image = cv2.imread(videopath + "/" + frame)
		imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
		grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
		frames.append(grayimage)
	frames = numpy.asarray(frames)
	videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
	training_list.append(videoarray)


directorylisting = os.listdir(disgustpath)
for video in directorylisting:
	frames = []
	videopath = disgustpath + video
	framerange = os.listdir(videopath)

	for frame in framerange:
		image = cv2.imread(videopath + "/" + frame)
		imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
		grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
		frames.append(grayimage)
	frames = numpy.asarray(frames)
	#print(frames)
	videoarray = numpy.rollaxis(numpy.rollaxis(frames, 2, 0), 2, 0)
	training_list.append(videoarray)

In [117]:
training_list = numpy.asarray(training_list)
trainingsamples = len(training_list)


traininglabels = numpy.zeros((trainingsamples, ), dtype = int)


traininglabels[0:76] = 0
traininglabels[76:170] = 1
traininglabels[170:206] = 2

traininglabels = np_utils.to_categorical(traininglabels, 3)

training_data = [training_list, traininglabels]
(trainingframes, traininglabels) = (training_data[0], training_data[1])
training_set = numpy.zeros((trainingsamples,1, image_rows, image_columns,  image_depth))
print(training_set.shape)
for h in range(trainingsamples):
	training_set[h][0][:][:][:] = trainingframes[h,:,:,:]

training_set = training_set.astype('float32')
training_set -= numpy.mean(training_set)
training_set /= numpy.max(training_set)
#print(training_set)
training_set=numpy.array(training_set)
training_set=tf.transpose(training_set,[0, 4 ,2 ,3 ,1])
print(training_set.shape)


(6, 1, 64, 64, 41)
(6, 41, 64, 64, 1)


In [118]:
# Save training images and labels in a numpy array
numpy.save('/content/drive/My Drive/miro/microexpstcnn_images.npy', training_set)
numpy.save('/content/drive/My Drive/miro/microexpstcnn_labels.npy', traininglabels)


# Load training images and labels that are stored in numpy array
training_set = numpy.load('/content/drive/My Drive/miro/microexpstcnn_images.npy')
traininglabels =numpy.load('/content/drive/My Drive/miro/microexpstcnn_labels.npy')

In [121]:
# MicroExpSTCNN Model
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(image_depth ,image_rows, image_columns, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))#init='normal',
model.add(Dropout(0.5))
model.add(Dense(3,))# init='normal'
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 39, 62, 50, 32)    4352      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 13, 20, 16, 32)    0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 13, 20, 16, 32)    0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 133120)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               17039488  
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)               

In [122]:
print(training_set.shape)

(6, 41, 64, 64, 1)


In [123]:
# Load pre-trained weights
"""
model.load_weights('/content/drive/My Drive/miro/weights-improvement-53-0.88.hdf5')
"""
filepath="/content/drive/My Drive/miro/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [128]:
# Spliting the dataset into training and validation sets
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_set, traininglabels, test_size=0.2, random_state=4)


# Save validation set in a numpy array

numpy.save('/content/drive/My Drive/miro/microexpstcnn_val_images.npy', validation_images)
numpy.save('/content/drive/My Drive/miro/microexpstcnn_val_labels.npy', validation_labels)


# Load validation set from numpy array

validation_images = numpy.load('/content/drive/My Drive/miro/microexpstcnn_val_images.npy')
validation_labels = numpy.load('/content/drive/My Drive/miro/microexpstcnn_val_labels.npy')


# Training the model
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), callbacks=callbacks_list, batch_size = 1, shuffle=True)#nb_epoch = 100,

# Finding Confusion Matrix using pretrained weights
"""
predictions = model.predict(validation_images)
predictions_labels = numpy.argmax(predictions, axis=1)
validation_labels = numpy.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)
"""

4/4 [==============================] - 3s 838ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000


KeyError: ignored